# Creation of an audio file with self-specified frequencies & intuition for the mathematical Fourier transform

##### Johannes Bahrenberg, 28.11.2023

In [7]:
# Standard libraries
import numpy as np
import matplotlib.pyplot as plt

# Slider for plots
from matplotlib.widgets import Slider

# Integration
from scipy import integrate

# Write audio file
from scipy.io.wavfile import write

In [8]:
samplerate = 44100    # samples / second

In [9]:
# Create random frequencies between 300 and 3700
#fs = np.floor(3700 * np.random.rand(nnumbers) + 300)
#fs = fs//10 * 10

# ALTERNATIVE:
# Create array of frequencies (best between 300 and 4000)
fs = np.array([400, 1100])

# Set time
tmax = 5   # length in seconds
t = np.linspace(0., tmax, tmax*samplerate)

In [10]:
# Create sound wave (=overlap of distinct freq. with same amplitude)
amplitude = np.iinfo(np.int16).max

data = np.zeros_like(t)
for f in fs:
    data += amplitude * np.cos(2. * np.pi * f * t)
data /= len(fs)

# Save file
write('soundwave.wav', samplerate, data.astype(np.int16))

In [11]:
def cosinus(f):
    '''Cosinus with frequency f.
    
    Note: t is a global variable (array) that defines the
    times at which the signal is recorded.
    Note: datamaxplot is a global variable that defines
    the index up to which the signal is depicted.
    '''
    return np.cos(2. * np.pi * f * t[:datamaxplot])

def integrand(t, f):
    '''The cos() part in the integrand. '''
    return np.cos(2. * np.pi * f * t)

In [13]:
%matplotlib qt

# Global variable for max. index (we don't plot the whole time interval)
datamaxplot = 1000

fig, ax = plt.subplots(2,1,figsize=(15,15))

# Investigate amplitude graph, i.e. air pressure wrt. time
ax[0].plot(t[:datamaxplot], data[:datamaxplot], color='0.2', label='signal')
ax[0].hlines(0, 0, t[datamaxplot], ls='--', color='black')
ax[0].set_title(f'Original signal\nFrequencies: {np.sort(fs)} Hz')

finit = 1000    # Initial frequency
p1, = ax[1].plot(t[:datamaxplot], integrand(t, finit)[:datamaxplot], 
        color = 'red', alpha=0.8)
p2, = ax[1].plot(t[:datamaxplot], amplitude * np.cos(2. * np.pi * finit * t[:datamaxplot]), 
        ls=':', color='red', alpha=0.6)
ax[1].hlines(0, 0, t[datamaxplot], ls='--', color='black')
ax[1].set_xlabel('t [s]')
ax[1].set_ylabel('amplitude')
time_text = ax[1].text(0., -30000, r'$\int_{t}dt \; f(t) \, cos(ft)$', fontsize=20, color='black')
attention_text = ax[1].text(0.008, -30000, '', fontsize=30, color='red')
time_text.set_bbox(dict(facecolor='white', edgecolor='black'))
attention_text.set_bbox(dict(facecolor='white', edgecolor='black'))

# make space for slider
plt.subplots_adjust(bottom=0.2)

# slider position
ax_slider = plt.axes([0.2, 0.1, 0.6, 0.02])    # xstart, ystart, xwidth, ywidth

# slider
slide = Slider(ax_slider, label='frequency f', valmin=300, valmax=3000, valinit=finit, valstep=10)

def update(val):
    # get current a factor
    current_f = slide.val
    
    # compute integral 
    integral = integrate.simpson(data[:10*datamaxplot]*integrand(t[:10*datamaxplot], current_f), t[:10*datamaxplot])
    
    # display integral solution
    time_text.set_text(f'$\int_{{t}}dt \; f(t) \, cos(ft) =$ {integral:.2f}')
    if np.isin(current_f, fs):
        attention_text.set_text('!')    # if we hit the correct frequency
    else:
        attention_text.set_text('')
    
    # compute the function and set y data
    function = cosinus(f=current_f)
    p1.set_ydata(data[:datamaxplot] * function)
    p2.set_ydata(amplitude * function)
    fig.canvas.draw()

# connect slider with update function
slide.on_changed(update)

plt.show()